In [ ]:
#7.3.2- Fine-Tuning T5 for Abstractive Summarization

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset

# Load dataset and model
dataset = load_dataset("xsum")
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Preprocessing
def preprocess_function(example):
    input_text = f"summarize: {example['document']}"
    target_text = example["summary"]
    model_inputs = tokenizer(input_text, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(target_text, max_length=150, truncation=True, padding="max_length").input_ids
    model_inputs["labels"] = labels
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Training setup
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="./t5-summarization", num_train_epochs=3, per_device_train_batch_size=8)
trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_dataset["train"], eval_dataset=tokenized_dataset["validation"])
trainer.train()


